descriptor<br>

keyword: descriptor, set, get, del, property <br>
1. 객체에서 서로 다른 객체를 속성값으로 가지는 것, Has-A relationship
2. read, write, delete 등을 미리 정의 가능
3. data descriptor(set, del), non-data descriptor(get)
4. 읽기 전용 객체 생성 장점, 클래스를 의도하는 방향으로 생성 가능

In [1]:
#ex1
# 기본적인 descriptor 예제

class DescriptorEx1():

    def __init__(self, name='Default'):
        self.name = name

    def __get__(self, obj, objtype):

        return f'get method called {self}, {obj}, {objtype}, {self.name}'

    def __set__(self, obj, name):
        print('set method called')

        if isinstance(name, str):
            self.name = name
        else:
            raise TypeError('Name should be str')

    def __delete__(self, obj):

        print('Delete method called')

        self.name = None
        
    # name = property(__get__, __set__, __delete__, '')


In [2]:
class Sample1():
    name = DescriptorEx1()
    # 위에서 만든 디스크립터 클래스 안에 있는 메소드가 알아서 호출된다.
    # 이전 getter, setter는 변수별로 하나씩 따로 만들었어야 했다.


In [3]:
ee1 = Sample1()
ee2 = Sample1()
ee1.name = '1'
ee2.name = '2'
print(ee1.name,'\n' ,ee2.name)
print(Sample1.name)

# 여기선 값이 연동된다.
# 조심해라 공유되냐 마냐 문제가 있다.

Sample1.name = 'duck'
print(ee1.name, ee2.name)



set method called
set method called
get method called <__main__.DescriptorEx1 object at 0x0000028A664FB9D0>, <__main__.Sample1 object at 0x0000028A672EE940>, <class '__main__.Sample1'>, 2 
 get method called <__main__.DescriptorEx1 object at 0x0000028A664FB9D0>, <__main__.Sample1 object at 0x0000028A672EECA0>, <class '__main__.Sample1'>, 2
get method called <__main__.DescriptorEx1 object at 0x0000028A664FB9D0>, None, <class '__main__.Sample1'>, 2
duck duck


In [4]:
s1 = Sample1()
s1.name = 'Descript test1'

print(s1.name)



Descript test1


In [5]:
# __delete__ 호출

del s1.name

print(f'ex1 > {s1.name}')

ex1 > duck


class property(fget=None, fset=None, fdel=None, doc=None)
(getter, setter, delete, doc: str)

In [6]:
# ex2
# property 클래스 사용 descriptor 직접 구현

# 구현 클래스와 사용 클래스가 동일함
class DescriptorEx2():

    def __init__(self, value):
        self._name = value

    def getVal(self):
        return f'get method called {self}, {self._name}'

    def setVal(self, value):

        print('set method called')
        if isinstance(value, str):
            self._name = value
        else:
            raise TypeError('Name should be str')

    def delVal(self):
        print('delete method called')
        self._name = None

        
    name = property(getVal, setVal, delVal, 'property method example')
    #property class 호출

    """
    name = property()에서 name 말고 name2 같이 다른 이름으로 바꾸면 진짜 attribute는
    name2라는 이름을 갖는다.
    
    
    """


In [7]:
ss1 = DescriptorEx2('1')
ss2 = DescriptorEx2('2')
print(ss1.name)
ss1.name = '8'
print(ss2.name)
print(ss1.name)

# 이 실험 결과에서 name은 class의 모든 객체가 공유하는 변수가 아니고 instance 변수임을 증명했다.

get method called <__main__.DescriptorEx2 object at 0x0000028A664FBBB0>, 1
set method called
get method called <__main__.DescriptorEx2 object at 0x0000028A664FBFD0>, 2
get method called <__main__.DescriptorEx2 object at 0x0000028A664FBBB0>, 8


In [8]:
# 최초값 확인

s2 = DescriptorEx2('Descriptor test2')

print(f'ex2 > {s2.name}')

s2.name = 'Descriptor test2 method'

print(f'ex2 > {s2.name}')

del s2.name

print(f'ex2 > {DescriptorEx2.name.__doc__}')



ex2 > get method called <__main__.DescriptorEx2 object at 0x0000028A672EEFA0>, Descriptor test2
set method called
ex2 > get method called <__main__.DescriptorEx2 object at 0x0000028A672EEFA0>, Descriptor test2 method
delete method called
ex2 > property method example


descriptor(low level) vs property(high level)

In [9]:
class DescriptorEx3():

    def __init__(self, value):
        self.name = value

    def getVal(self):
        return f'get method called {self}, {self.name}'

    def setVal(self, value):

        print('set method called')
        if isinstance(value, str):
            self.name = value
        else:
            raise TypeError('Name should be str')

    def delVal(self):
        print('delete method called')
        self.name = None

        
    name2 = property(getVal, setVal, delVal, 'property method example')
    

In [10]:
# 최초값 확인

s3 = DescriptorEx3('Descriptor test3')

print(f'ex3 > {s3.name}')

s3.name = 'Descriptor test3 method'

print(f'ex3 > {s3.name}')

del s3.name



# 내가 만든 getter, setter가 호출되는 것이 아니다.
# init이 만든 name과 property의 name2가 따로 있는 것이다.

ex3 > Descriptor test3
ex3 > Descriptor test3 method


디스크립터
1. 상황에 맞는 메소드 구현을 통한 객체 지향 프로그래밍 구현
2. property와 달리 reuse(재사용) 가능
3. ORM Framework에서 사용


In [11]:
# ex1

import os

class DirectoryFileCount:
    
    def __get__(self, obj, objtype=None):
        #print(os.listdir(obj.dirname))
        return len(os.listdir(obj.dirname))


class DirectoryPath:
    # descriptor instance

    size = DirectoryFileCount()

    def __init__(self, dirname):
        self.dirname = dirname

    

In [12]:
# 현재 경로

s = DirectoryPath('./')

g = DirectoryPath('../')

print(s.size, g.size)



22 13


In [13]:
print(f'ex1 > {dir(DirectoryPath)}')
print(f'ex1 > {dir(DirectoryPath.__dict__)}')

ex1 > ['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'size']
ex1 > ['__class__', '__contains__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', 'copy', 'get', 'items', 'keys', 'values']


In [14]:
print(f'ex1 > {dir(s)}')

print(f'ex1 > {s.__dict__}')


ex1 > ['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'dirname', 'size']
ex1 > {'dirname': './'}


In [15]:
# ex2
# log

import logging

logging.basicConfig(
    format='%(asctime)s %(message)s',
    level = logging.INFO,
    datefmt = '%Y-%m-%d %H:%M:%S'
)

class LoggedScoreAccess:

    def __init__(self, value=50):
        self.value = value

    def __get__(self, obj, objtype=None):

        logging.info('Accessing %r giving %r', 'score', self.value)
        return self.value

    def __set__(self, obj, value):
        logging.info('Updating %r giving %r', 'score', self.value)
        self.value = value

class Student:
    # descriptor instance
    score = LoggedScoreAccess()

    def __init__(self, name):
        self.name = name
        # regular instance attribute



In [16]:
s1 = Student('Kim')
s2 = Student('Lee')

print(f'ex2 > {s1.score}')

2022-08-07 23:49:28 Accessing 'score' giving 50


ex2 > 50


In [17]:
s1.score += 20 # get -> set 차례로 실행되는 코드다.
print(f'ex2 > {s1.score}')


2022-08-07 23:49:28 Accessing 'score' giving 50
2022-08-07 23:49:28 Updating 'score' giving 50
2022-08-07 23:49:28 Accessing 'score' giving 70


ex2 > 70


In [18]:
# __dict__

print(f'ex2 > {vars(s1)}')
print(f'ex2 > {s1.__dict__}')

ex2 > {'name': 'Kim'}
ex2 > {'name': 'Kim'}
